# `Dataset 구축 작업`

- ### by : SY
- ### date : Feb 17, 2023

In [1]:
import torch
torch.cuda.is_available()

True

In [263]:
#import modules
import glob
import shutil
import os
import zipfile
import re 
import pandas as pd

In [374]:
#assign dir
directory =os.getenv('HOME') + '/aiffel/aiffelthon/'

# make directory for zipfiles ->
zipfiles = [i[26:] for i in glob.glob(directory + '*.zip')] 
for i,r in enumerate(zipfiles) :
    os.mkdir(directory + f'reg_{i+1}' )

In [380]:
# unzip files
for i, file in enumerate(zipfiles) :
    with zipfile.ZipFile(directory + file, 'r') as zip_ref:
        zip_ref.extractall(directory+f'reg_{i+1}')

In [382]:
def doc_preprocess(txt,reg):
    raw = []
    for doc in txt : 
        with open(doc,'r') as f :
            lines = f.readlines() # 한줄 한줄씩 파일 가져오기
            for line in lines :
                line = re.sub(r'\d\W','',line)
                match = re.search(r'\(\S+\)\/\(\S+\)',line) # 찾기 조건
                if match :
                    line = line.strip() # \n 부호 지우기
                    pair = ' '.join(re.findall(r'\(\S+\)\/\(\S+\)',line))
                    normal = re.sub(r'\(\S+\)\/','', line) #표준어
                    dialect = re.sub(r'\/\(\S+\)','', line) #사투리
                    raw.append((normal, dialect,reg,pair)) #( 표준어, 사투리, 지역tag, original 표준어/사투리 )
                    
    return raw

In [383]:
for i,r in enumerate(zipfiles) :
    print(i, r)

0 [라벨]제주도_학습용데이터_1.zip
1 [라벨]전라도_학습데이터_1.zip
2 [라벨]경상도_학습데이터_1.zip
3 [라벨]충청도_학습데이터_1.zip
4 [라벨]강원도_학습데이터_1.zip


In [384]:
data = {} # dictionary 형태로 저장
for i in range(len(zipfiles)) :
    txt = glob.glob(directory+ f'reg_{i+1}'+ '/*.txt')
    data[f'reg_{i+1}'] = doc_preprocess(txt, i+1)
    print(f'total no. files reg_{i+1} : {len(txt)}\n')

total no. files reg_1 : 5042

total no. files reg_2 : 7412

total no. files reg_3 : 7699

total no. files reg_4 : 6048

total no. files reg_5 : 4717



In [386]:
#seq_len 최대, 최소, average 구하기

def find_min_max(data) :
    max_len = 0
    min_len = 999
    av_len = 0
    for i in data :
        length = len(i[1].split())
        if max_len < length : max_len = length
        if min_len > length : min_len = length
        av_len += length
    print(f' min: {min_len}, max :{max_len}, avg : {av_len/len(data)}, seq_total :{len(data)}')
    
for k,v in data.items() :
    print(k)
    find_min_max(v)

reg_1
 min: 1, max :88, avg : 6.320686577165759, seq_total :1172949
reg_2
 min: 1, max :39, avg : 8.828960198263838, seq_total :295364
reg_3
 min: 1, max :67, avg : 9.814914272059262, seq_total :334780
reg_4
 min: 1, max :43, avg : 10.541839715294238, seq_total :275934
reg_5
 min: 1, max :56, avg : 5.642623380738782, seq_total :915155


In [387]:
a = pd.DataFrame(columns = ['text','dial','reg','pair'])

for i in data.values() :
    b = pd.DataFrame(i, columns = ['text','dial','reg','pair'])
    a = a.merge(b,how='outer')

a.to_csv(directory + '/dataset_full_v2.csv',index=False)

a.info()
a['reg'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2994182 entries, 0 to 2994181
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
 1   dial    object
 2   reg     int64 
 3   pair    object
dtypes: int64(1), object(3)
memory usage: 114.2+ MB


1    1172949
5     915155
3     334780
2     295364
4     275934
Name: reg, dtype: int64

In [337]:
# # remove all files
# for i in des : 
#     shutil.rmtree(os.path.join(directory,i))